In [1]:
import torch
import torch.nn as nn

# Example of a convolutional layer
conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)

UP SAMPLING

In [2]:
import torch.nn.functional as F

# Example of up sampling
x = torch.randn(1, 16, 32, 32)  # Example input tensor
upsampled_x = F.interpolate(x, scale_factor=2, mode='nearest')  # Upsample by a factor of 2 (nearest: duplicating the existing pixels or samples to create new ones)


In [3]:
import numpy as np
array = np.array([[[1,2],[2,3]],[[4,5],[6,7]], [[8,9],[10,11]]], dtype=np.float64)
print(array, array.shape)
interpolated = F.interpolate(torch.from_numpy(array), scale_factor=2, mode='nearest')
print(interpolated, interpolated.shape)

[[[ 1.  2.]
  [ 2.  3.]]

 [[ 4.  5.]
  [ 6.  7.]]

 [[ 8.  9.]
  [10. 11.]]] (3, 2, 2)
tensor([[[ 1.,  1.,  2.,  2.],
         [ 2.,  2.,  3.,  3.]],

        [[ 4.,  4.,  5.,  5.],
         [ 6.,  6.,  7.,  7.]],

        [[ 8.,  8.,  9.,  9.],
         [10., 10., 11., 11.]]], dtype=torch.float64) torch.Size([3, 2, 4])


DOWNSAMPLING

In [4]:
# Example of downsampling using max pooling
max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
downsampled_x = max_pool(x)  # Down sample by a factor of 2
print(downsampled_x.shape)

torch.Size([1, 16, 16, 16])


BATCH NORMALIZATION

In [5]:
# Example of batch normalization
batch_norm = nn.BatchNorm2d(num_features=16)
normalized_x = batch_norm(upsampled_x)


ACTIVATION FUNCTIONS

In [6]:
# Example of ReLU activation function
relu = nn.ReLU()
activated_x = relu(normalized_x)


In [7]:
activated_x.shape

torch.Size([1, 16, 64, 64])

LINEAR LAYERS

In [8]:
# Example of a linear layer
linear_layer = nn.Linear(in_features=65536, out_features=10)
output = linear_layer(activated_x.view(activated_x.size(0), -1))  # Flatten the input before feeding into the linear layer

CREATING THE NEURAL NETWORK

1. DEFINE NETWORK

In [10]:
import torch.nn as nn
import torch.nn.functional as F
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm  # For progress bar
import random


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.batch_norm1 = nn.BatchNorm2d(16)
        self.batch_norm2 = nn.BatchNorm2d(32)

    def forward(self, x):
        x = self.pool(F.relu(self.batch_norm1((self.conv1(x)))))
        print(x.shape)
        x = self.pool(F.relu(self.batch_norm2(self.conv2(x))))
        print(x.shape)
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = SimpleCNN()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. DEFINE LOSS FUNCTION AND OPTIMIZER

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


3. TRAIN THE NETWORK

In [12]:
def train(model, train_loader, criterion, optimizer, epochs=1):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Example usage (assuming `train_loader` is defined)
# train(model, train_loader, criterion, optimizer)


4. TEST THE NETWORK

In [13]:
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total}%")

# Example usage (assuming `test_loader` is defined)
# test(model, test_loader)


TRAINING WITH OUR DATA

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [24]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.batch_norm1 = nn.BatchNorm2d(16)
        self.batch_norm2 = nn.BatchNorm2d(32)

    def forward(self, x):
        x = self.pool(F.relu(self.batch_norm1((self.conv1(x)))))
        print(x.shape)
        x = self.pool(F.relu(self.batch_norm2(self.conv2(x))))
        print(x.shape)
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = SimpleCNN()

In [25]:
class APTOSDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, 'train_images', self.data_frame.iloc[idx, 0] + '.png')
        image = Image.open(img_name).convert('RGB')
        label = self.data_frame.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        return image, label





In [26]:
data_path = '/content/drive/My Drive/aptos2019-blindness-detection'
csv_file = os.path.join(data_path, 'train.csv')

In [27]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = APTOSDataset(csv_file = csv_file, root_dir = data_path, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers = 2)

validset = APTOSDataset(csv_file = csv_file, root_dir = data_path, transform=transform)
validloader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=False, num_workers=2)

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [29]:
def train(model, train_loader, valid_loader, criterion, optimizer, epochs=1):
    train_losses, valid_losses = [], []
    train_accuracies, valid_accuracies = [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        valid_loss, valid_accuracy = validate(model, valid_loader, criterion)
        valid_losses.append(valid_loss)
        valid_accuracies.append(valid_accuracy)

        print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}%, Valid Loss: {valid_loss}, Valid Accuracy: {valid_accuracy}%")

    return train_losses, valid_losses, train_accuracies, valid_accuracies

def validate(model, valid_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    valid_loss = running_loss / len(valid_loader)
    valid_accuracy = 100 * correct / total
    return valid_loss, valid_accuracy

# Train the model
train_losses, valid_losses, train_accuracies, valid_accuracies = train(model, trainloader, validloader, criterion, optimizer, epochs=5)

torch.Size([64, 16, 112, 112])
torch.Size([64, 32, 56, 56])


ValueError: Expected input batch_size (4096) to match target batch_size (64).

In [ ]:
# Plotting training and validation loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Valid Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plotting training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(valid_accuracies, label='Valid Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()
